# Remote Sensing with Python

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

*Stewart Udall, Secretary of the Interior, September 21, 1966*

### The Workflow

We will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest) in the Campania region
- import multiple Sentinel images from the dates before and after the fires
- determine which images are best for analysis
- create various NDVI map outputs to assess the amount of fire damage

[2017 Mt. Vesuvius Fire (in Italian)](https://www.vesuviolive.it/ultime-notizie/256710-11-luglio-2017-a-fuoco-il-vesuvio-e-il-piu-grande-disastro-ambientale-della-sua-storia/)

We will use Google Earth Engine's "Harmonized Sentinel-2 MSI: MultiSpectral Instrument, Level-2A"

- [EE Sentinel 2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

In [1]:
from datetime import datetime

import ee
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import Image

from utils import add_ee_layer

In [2]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize()

## Define filters

In [3]:
# coordinates of Vesuvius
lat = 40.821330
lon = 14.426102

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start and end dates of range to filter for
start_date, end_date = '2017-05-10', '2017-08-10'

## Get Sentinel 2 data

In [4]:
# get the satellite data
sentinel = ee.ImageCollection(
    "COPERNICUS/S2_SR_HARMONIZED"
).filterBounds(poi).filterDate(start_date,end_date)

In [5]:
# how many images did we get?
print('Total number:', sentinel.size().getInfo())

Total number: 10


In [6]:
# information about the first image in our collection
# sentinel.first().getInfo()

In [7]:
# what about cloud cover of our first image?
# sentinel.first().get('CLOUD_COVERAGE_ASSESSMENT').getInfo()

In [8]:
# what bands did we get?
# sentinel.first().bandNames().getInfo()

In [9]:
# put the images in a list
sentinel_list = sentinel.toList(sentinel.size());

## Display satellite image

- [What are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine)

In [10]:
# set some parameters for the images
parameters = {
                'min': 0,
                'max': 3_000,
                'dimensions': 1_000, # square size in pixels
                'bands': ['B4', 'B3', 'B2'] # bands to display (r,g,b)
             }

In [11]:
data = []

for i in range(sentinel.size().getInfo()):

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')
    
    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVERAGE_ASSESSMENT').getInfo()
    
    print('Image #', i, date_time, 'Cloud cover:', cloud)
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

    this_data = [i, date_time, cloud]
    data.append(this_data)    

df = pd.DataFrame(data, columns = ['Image #', 'Datetime', 'Cloud Cover'])

Image # 0 2017-05-20 07:36:08 Cloud cover: 0.6981


Image # 1 2017-05-30 08:23:56 Cloud cover: 13.670099999999998


Image # 2 2017-06-09 23:50:06 Cloud cover: 26.7629


Image # 3 2017-06-10 02:12:06 Cloud cover: 23.1136


Image # 4 2017-07-06 05:42:35 Cloud cover: 17.0598


Image # 5 2017-06-29 08:00:24 Cloud cover: 11.981


Image # 6 2017-07-14 20:17:10 Cloud cover: 0.2403


Image # 7 2017-07-19 10:25:54 Cloud cover: 9.0137


Image # 8 2017-07-29 14:04:20 Cloud cover: 5.4445


Image # 9 2017-08-17 12:06:20 Cloud cover: 0


In [12]:
df

,Image #,Datetime,Cloud Cover
0,0,2017-05-20 07:36:08,0.6981
1,1,2017-05-30 08:23:56,13.6701
2,2,2017-06-09 23:50:06,26.7629
3,3,2017-06-10 02:12:06,23.1136
4,4,2017-07-06 05:42:35,17.0598
5,5,2017-06-29 08:00:24,11.9810
6,6,2017-07-14 20:17:10,0.2403
7,7,2017-07-19 10:25:54,9.0137
8,8,2017-07-29 14:04:20,5.4445
9,9,2017-08-17 12:06:20,0.0000


## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by applying a buffer around our POI.

In [13]:
# create a list of images we want (before, during, after)
sentinel_sequence = [0,9]

In [14]:
# Define a region of interest with a buffer zone of 6 km (6_000 metres)
roi = poi.buffer(5_000)

In [15]:
parameters = {
                'min': 0,
                'max': 3_000,
                'dimensions': 2_800,
                'bands': ['B4', 'B3', 'B2'],
                'region': roi
             }

In [16]:
for i in sentinel_sequence:
    
    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    # cloud cover
    cloud = ee.Image(sentinel_list.get(i)).get('CLOUD_COVERAGE_ASSESSMENT').getInfo()
    
    print('Image #', i, date_time, 'Cloud cover:', cloud)
    
    display(Image(url = ee.Image(sentinel_list.get(i)).getThumbUrl(parameters)))

Image # 0 2017-05-20 07:36:08 Cloud cover: 0.6981


Image # 9 2017-08-17 12:06:20 Cloud cover: 0


## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

- [Source: Agricolus](https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/)

In [17]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.3,
                   'dimensions': 2_850,
                   'palette': palette,
                   'region': roi}

In [18]:
for i in sentinel_sequence:

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    print('Image #', i, date_time)
    
    # display the image
    display(
        Image(
            url=ee.Image(sentinel_list.get(i)).normalizedDifference(['B8', 'B4']).getThumbUrl(ndvi_parameters)
        )
    )

Image # 0 2017-05-20 07:36:08


Image # 9 2017-08-17 12:06:20


In [19]:
folium.Map.add_ee_layer = add_ee_layer
vesuvius_map = folium.Map(location=[lat, lon], zoom_start=13)

# Add a layer for each satellite image of interest (before, during and after)
for i in sentinel_sequence:

    # when was this image taken?
    unix_time = ee.Image(sentinel_list.get(i)).get('GENERATION_TIME').getInfo() / 1_000
    date_time = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')

    vesuvius_map.add_ee_layer(
        ee.Image(sentinel_list.get(i)).normalizedDifference(['B8', 'B4']), 
        ndvi_parameters,
        name=date_time,
    )
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(vesuvius_map)

# vesuvius_map.save('vesuvius.html')
display(vesuvius_map)